# SGDRegressor 

Nessa regressão foi previsto o impacto das notas (de críticos, usuários e Vgchartzscore) e também do genero do jogo (ação, tiro, aventura, ect) nas vendas totais desse jogo, assim prevendo qual vai ser o proximo genenro mais vendido e quais serão suas notas.

In [5]:
%matplotlib notebook
%reset -f


import pandas as pd
import numpy as np
from scipy.stats import norm, probplot
import statsmodels.api as sm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from sklearn import preprocessing

In [62]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [63]:
df = pd.read_excel("jogo.xlsx")

In [64]:
df = df.fillna(0)

In [9]:
deletar = df.loc[((df.Global_Sales.fillna(0)) == 0) & ((df.Total_Shipped.fillna(0)) == 0)]
a = deletar.index
dados = df.drop(a)

In [10]:
dados["Sales"] = dados["Global_Sales"].fillna(0) + dados["Total_Shipped"].fillna(0)

In [11]:
dados = dados.drop(columns = ["Global_Sales","Total_Shipped","url","status","img_url","NA_Sales" ,"PAL_Sales","JP_Sales" ,"Other_Sales","VGChartz_Score"])


In [12]:
limpa = dados.loc[((dados.Sales != 0 ) & (dados.Critic_Score != 0))]
limpa

,Rank,Name,basename,Genre,ESRB_Rating,Platform,Publisher,Developer,Critic_Score,User_Score,Year,Last_Update,Vgchartzscore,Sales
0,1,Wii Sports,wii-sports,Sports,E,Wii,Nintendo,Nintendo EAD,7.7,0.0,2006.0,0,0.0,82.86
1,2,Super Mario Bros.,super-mario-bros,Platform,0,NES,Nintendo,Nintendo EAD,10.0,0.0,1985.0,0,0.0,40.24
2,3,Mario Kart Wii,mario-kart-wii,Racing,E,Wii,Nintendo,Nintendo EAD,8.2,9.1,2008.0,11th Apr 18,8.7,37.14
4,5,Wii Sports Resort,wii-sports-resort,Sports,E,Wii,Nintendo,Nintendo EAD,8.0,8.8,2009.0,0,8.8,33.09
5,6,Pokemon Red / Green / Blue Version,pokmon-red,Role-Playing,E,GB,Nintendo,Game Freak,9.4,0.0,1998.0,0,0.0,31.38
6,7,New Super Mario Bros.,new-super-mario-bros,Platform,E,DS,Nintendo,Nintendo EAD,9.1,8.1,2006.0,0,0.0,30.80
8,9,New Super Mario Bros. Wii,new-super-mario-bros-wii,Platform,E,Wii,Nintendo,Nintendo EAD,8.6,9.2,2009.0,0,9.1,30.22
9,10,Minecraft,minecraft,Misc,0,PC,Mojang,Mojang AB,10.0,0.0,2010.0,05th Aug 18,0.0,30.01
11,12,Wii Play,wii-play,Misc,E,Wii,Nintendo,Nintendo EAD,5.9,4.5,2007.0,0,5.3,28.02
12,13,Kinect Adventures!,kinect-adventures,Party,E,X360,Microsoft Game Studios,Good Science Studio,6.7,0.0,2010.0,29th Sep 18,6.2,24.00


In [153]:

from sklearn.linear_model import SGDRegressor

In [154]:
fitter = SGDRegressor(loss="squared_loss", penalty=None)

In [155]:

def dummify(data, column_name):
    """
        Converte a coluna column_name em dummies / one-hot e as adiciona ao dataframe
        retorna uma copia do  df original *sem* a coluna que foi dummified
    """
    df = data.copy()
    # Nota: se você for um econometrista, pode se interessar por usar a opção dropFirst = True na get_dummies
    df2 = pd.concat([df.drop(column_name, axis=1), pd.get_dummies(data[column_name], prefix=column_name)], axis=1)
    return df2

In [175]:
data_dum=dummify(limpa[["Critic_Score", "User_Score","Vgchartzscore","Genre","Sales"]],"Genre")
len(data_dum.columns)

24

In [182]:
lista_nova = list(data_dum.columns)
lista_nova.remove("Sales")
data_trei = data_dum[lista_nova]
sales = data_dum["Sales"]

In [183]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_trei,sales, test_size=0.25, random_state=RANDOM_SEED)



In [184]:
X = X_train.as_matrix(columns=list(X_train.columns)) 


C:\Users\pedro\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [185]:
X

array([[7.9, 0. , 0. , ..., 0. , 0. , 0. ],
       [9. , 0. , 0. , ..., 0. , 0. , 0. ],
       [8.4, 8.8, 0. , ..., 0. , 0. , 0. ],
       ...,
       [7.8, 0. , 0. , ..., 0. , 0. , 0. ],
       [7.3, 0. , 8. , ..., 0. , 0. , 0. ],
       [4.8, 4.8, 0. , ..., 0. , 0. , 0. ]])

In [194]:
y = pd.Series(y_train).tolist()

In [195]:
fitter.fit(X, y)

C:\Users\pedro\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
       eta0=0.01, fit_intercept=True, l1_ratio=0.15,
       learning_rate='invscaling', loss='squared_loss', max_iter=None,
       n_iter=None, n_iter_no_change=5, penalty=None, power_t=0.25,
       random_state=None, shuffle=True, tol=None, validation_fraction=0.1,
       verbose=0, warm_start=False)

In [226]:
fitter.score(X_test,y_test)

-0.2599900333069647

In [209]:
resultado=fitter.predict(X_test)

In [229]:
lista=[]
for e in resultado:
    if (e * 0.10)+e in y_test or (e*0.10)-e in y_test:
        lista.append(1)
    else:
        lista.append(0)
print(lista)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 